# Label cell types using Seurat Label Transfer

To build our reference, we would like to start with labels that originate from published cell type references. 

One of the approaches for this cell type labeling is Seurat, which labels cells by integration with a reference dataset.  

Label transfer using Seurat is described [on their website](https://satijalab.org/seurat/articles/integration_mapping), and was introduced in this publication:  

Stuart, T. et al. Comprehensive Integration of Single-Cell Data. Cell 177, 1888–1902.e21 (2019)

Here, we'll load in our cells in batches, and assign cell types based on the PBMC reference dataset provided by the Satija lab as part of their 2021 publication in Cell (described below).

## Load libraries

`dplyr`: Data frame manipulation tools  
`H5weaver`: A package for reading .h5 files generated by AIFI  
`hise`: The HISE SDK  
`parallel`: Parallelization of processes in R  
`purrr`: Functional programing tools  
`Seurat`: Single-cell data analysis tools  
`SeuratObject`: Data structures for Seurat

We also set the `timeout` option to be high so that R waits for us to download the large reference dataset from Zenodo, below

In [3]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }

quiet_library(dplyr)
quiet_library(H5weaver)
quiet_library(hise)
quiet_library(parallel)
quiet_library(purrr)
quiet_library(Seurat)
quiet_library(SeuratObject)

options(timeout = 10000)

## Prepare Seurat PBMC reference

To use Seurat to label our cells, we'll utilize the PBMC reference provided by the Satija Lab for PBMCs generated using Seurat V5. This reference is derived from data in this publication from the Satija lab:

Hao, Y. and Hao, S. et al. Integrated analysis of multimodal single-cell data. Cell 184, 3573–3587.e29 (2021)

The version of record for this reference dataset is provided in a Zenodo repository at this accession:  
https://zenodo.org/records/7779017

Additional information about the cell type labels in this reference is available [on the Azimuth website](https://azimuth.hubmapconsortium.org/references/#Human%20-%20PBMC).

We'll download the reference from Zenodo for label transfer:

In [4]:
if(!dir.exists("reference")) {
    dir.create("reference")
}
download.file(
    "https://zenodo.org/records/7779017/files/pbmc_multimodal_2023.rds?download=1",
    "reference/pbmc_multimodal_2023.rds"
)

In [5]:
reference <- readRDS("reference/pbmc_multimodal_2023.rds")

Between Level 2 (L2) and Level 3 (L3) labels provided by the Satija lab, we like to add an additional level that we call L2.5, which separates the Treg Naive and Memory cells based on L3, and assigns a CD8 TEMRA cell label to cells with the L3 labels CD8 TEM_4 and CD8 TEM_5. All other cell types use their L2 assignments.

In [6]:
l3 <- as.character(reference@meta.data$celltype.l3)
l2 <- as.character(reference@meta.data$celltype.l2)
l2.5 <- l2
l2.5[l3 == "Treg Naive"] <- "Treg Naive"
l2.5[l3 == "Treg Memory"] <- "Treg Memory"
l2.5[l3 %in% c("CD8 TEM_4", "CD8 TEM_5")] <- "CD8 TEMRA"

reference <- AddMetaData(reference, metadata = l2.5, col.name = "celltype.l2.5")

## Retreive sample metadata

In an earlier step, we assembled and stored sample metadata in HISE. We'll pull this file, and use it to retrieve file for our labeling process.

In [7]:
sample_meta_uuid <- "223b4aa9-19fc-41e1-8bea-43682e5ac278"

In [8]:
res <- cacheFiles(list(sample_meta_uuid))
sample_meta_file <- list.files(
    paste0("cache/", sample_meta_uuid), 
    pattern = ".csv",
    full.names = TRUE
)

ERROR: Error in if (thisDesc$file$id != idsExpanded[[fidx]]) {: argument is of length zero


In [ ]:
hise_meta <- read.csv(sample_meta_file)

In [9]:
hise_meta <- read.csv("cache/downloadable/ref_h5_meta_data_2024-02-08.csv")

## Divide data into chunks for parallel processing

For labeling, we'll take files in batches of up to 10 files. We'll label those files, then output the results for each sample in the batch.

In [10]:
hise_meta <- hise_meta %>% 
  arrange(file.batchID)

In [11]:
b <- rep(1:11, each = 10)[1:nrow(hise_meta)]
df_chunk_list <- split(hise_meta, b)

In [12]:
map_int(df_chunk_list, nrow)

1  2  3  4  5  6  7  8  9 10 11 
10 10 10 10 10 10 10 10 10 10  8

## Prepare output directory

We'll store results for each sample in `output/Hao_PBMC/`.

In [13]:
out_dir <- "output/Hao_PBMC/"
if(!dir.exists(out_dir)) {
    dir.create(out_dir, recursive = TRUE)
}

## Functions for parallel label transfer of chunks

For each chunk, we'll retrieve the data from HISE, perform label transfer and ADT imputation using Seurat, and then store the labels and imputed ADT matrices to allow us to assess cell type identity.

The main function to perform these steps is `label_chunk()`, provided below. There are also 4 helper functions that assist in performing these steps for each sample:  
`read_so()` Reads the .h5 files stored in HISE as Seurat Objects for analysis  
`write_labels()` Writes the labeling results to .csv files for each sample  
`get_sample_adt()` subsets the ADT matrix for each sample and returns ADT values per sample  
`write_adt()` Writes the imputed ADT matrix values to a .h5 file for later use

In [14]:
read_so <- function(h5_uuid) {
    res <- cacheFiles(list(h5_uuid))
    h5_file <- list.files(
        paste0("cache/", h5_uuid), 
        pattern = ".h5", 
        full.names = TRUE
    )
    
    counts <- read_h5_dgCMatrix(h5_file)
    rownames(counts) <- make.unique(rownames(counts))
    meta <- read_h5_cell_meta(h5_file)
    rownames(meta) <- meta$barcodes

    so <- CreateSeuratObject(
      counts,
      meta.data = meta,
      assay = "RNA")

    rm_command <- paste0("rm -r cache/", h5_uuid)
    system(rm_command)

    return(so)
}

write_labels <- function(sample_labels, sample_id) {
    out_file <- paste0("output/Hao_PBMC/", sample_id, "_Hao_PBMC.csv")
    write.csv(
        sample_labels,
        out_file,
        row.names = FALSE,
        quote = FALSE
    )
}

get_sample_adt <- function(sample_meta, adt_data) {
    adt_data[,sample_meta$barcodes]
}

write_adt <- function(sample_adt, sample_id) {
    out_file <- paste0("output/Hao_PBMC/", sample_id, "_ADT.h5")
    list_mat <- list(
        i = sample_adt@i,
        p = sample_adt@p,
        x = sample_adt@x,
        Dim = dim(sample_adt),
        rownames = rownames(sample_adt),
        colnames = colnames(sample_adt)
    )

    h5createFile(out_file)
    h5write(list_mat, out_file, "mat")
}

label_chunk <- function(meta_data) {
    # Read and combine individual samples
    so_list <- map(meta_data$file.id, read_so)
    combined <- Reduce(merge, so_list)
    rm(so_list)

    # Transform data to match the reference
    combined <- SCTransform(
        combined,
        method = "glmGamPoi", 
        verbose = FALSE
    )
    
    # find anchors
    anchors <- FindTransferAnchors(
      reference = reference,
      query = combined,
      normalization.method = "SCT",
      reference.reduction = "spca",
      dims = 1:50
    )  
        
    #perform projection to get labels
    combined <- MapQuery(
      anchorset = anchors,
      query = combined,
      reference = reference,
      refdata = list(
        celltype.l1 = "celltype.l1",
        celltype.l2 = "celltype.l2",
        celltype.l3 = "celltype.l3",
        celltype.l2.5 = "celltype.l2.5",
        predicted_ADT = "ADT"
      ),
      reference.reduction = "spca", 
      reduction.model = "wnn.umap"
    )

    # Split the metadata by sample for output
    sample_meta_list <- split(combined@meta.data, combined@meta.data$pbmc_sample_id)

    # Write labels
    walk2(
        sample_meta_list, names(sample_meta_list),
        write_labels
    )

    # Subset and write projected ADT data
    sample_adt_list <- map(
        sample_meta_list, 
        get_sample_adt, 
        adt_data = combined@assays$predicted_ADT@data)

    walk2(
        sample_adt_list, names(sample_meta_list),
        write_adt
    )
}

## Apply label transfer to all chunks

We'll use `mclapply()` to perform our label transfer steps in parallel for multiple chunks. 

Because of the extremely memory-intensive use of `SCTransform()`, we're limited in the number of chunks that we can process simultaneously.

In [16]:
res <- mclapply(
    df_chunk_list,
    label_chunk,
    mc.cores = 3
)

$`1`
NULL

$`2`
NULL

$`3`
NULL

## Assemble results

Next, we'll assemble all of the labeling results in a single file for storage in HISE and downstream utilization.

In [18]:
label_files <- list.files(
    out_dir,
    pattern = ".csv",
    full.names = TRUE
)

In [20]:
all_labels <- map_dfr(label_files, read.csv)

In [22]:
all_labels <- all_labels %>%
  select(pbmc_sample_id, barcodes, starts_with("predicted"))

In [24]:
out_file <- paste0(
    "output/ref_seurat_labels_PBMC_",
    Sys.Date(),
    ".csv"
)
write.csv(
    all_labels,
    out_file,
    row.names = FALSE,
    quote = FALSE
)

## Store results in HISE

Finally, we store the output file in our Collaboration Space for later retrieval and use. We need to provide the UUID for our Collaboration Space (aka `studySpaceId`), as well as a title for this step in our analysis process.

The hise function `uploadFiles()` also requires the FileIDs from the original fileset for reference.

In [39]:
study_space_uuid <- "64097865-486d-43b3-8f94-74994e0a72e0"
title <- "Reference Seurat Label Predictions"

In [41]:
in_list <- list(c(sample_meta_uuid, hise_meta$file.id))

[[1]]
[1] "output/all_lm_dep_2023-10-09.csv"

In [40]:
out_list <- list(out_file)

In [42]:
uploadFiles(
    files = out_list,
    studySpaceId = study_space_uuid,
    title = title,
    inputFileIds = file_uuids,
    store = "project",
    doPrompt = FALSE
)

$files
$files[[1]]
[1] "output/all_lm_dep_2023-10-09.csv"


$traceId
[1] "01faca6e-1ef9-4d63-8438-01bb42417dcc"

In [17]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] Seurat_5.0.1       SeuratObject_5.0.1 sp_2.1-2           purrr_1.0.2       
 [5] furrr_0.3.1        future_1.33.1      dplyr_1.1.4        hise_2.16.0       
 [9] H5weaver_1.2.0     rhdf5_2.46.1       Matrix_1.6-4       data.table_1.14.10

loaded via a namespace (and not attached):
  [1] RColor